In [52]:
import pandas as pd
import numpy as np
import tensorflow as tf 
import torch
from torch.utils.data import TensorDataset, DataLoader
from pytorch_tcn import TCN
from keras import layers, models



Get Data

In [47]:
df = pd.read_csv("./simulated_dataset/all_data.csv")

df_pivot = df.pivot(index="timestamp", columns="sensor_id", values=["temperature", "label"])
df_pivot = df_pivot.reset_index(drop=True)

df_pivot.head()

sensor_dfs = {}

for i in range(1, 5):
    sensor_df = df_pivot.xs(i, axis=1, level=1).copy()
    
    sensor_dfs[i] = sensor_df

sensor_dfs


{1:        temperature  label
 0        47.088848    0.0
 1        47.939854    0.0
 2        43.477069    0.0
 3        44.807002    0.0
 4        49.098493    0.0
 ...            ...    ...
 21596    48.194567    0.0
 21597    50.989738    0.0
 21598    47.142103    0.0
 21599    44.380345    0.0
 21600    45.995404    0.0
 
 [21601 rows x 2 columns],
 2:        temperature  label
 0        51.786915    0.0
 1        49.126866    0.0
 2        49.720835    0.0
 3        49.667801    0.0
 4        57.302099    0.0
 ...            ...    ...
 21596    43.121297    0.0
 21597    49.984367    0.0
 21598    42.068131    0.0
 21599    49.353846    0.0
 21600    50.159115    0.0
 
 [21601 rows x 2 columns],
 3:        temperature  label
 0        56.962859    0.0
 1        53.570960    0.0
 2        51.641507    0.0
 3        58.683527    0.0
 4        50.081191    0.0
 ...            ...    ...
 21596    51.168246    0.0
 21597    52.867915    0.0
 21598    47.667152    0.0
 21599    52.14

Train/Test Split

In [48]:
train_dfs = {}
test_dfs = {}

for i, sensor_df in sensor_dfs.items():
    n = int(len(sensor_df) * .8)
    train_dfs[i] = sensor_df.iloc[:n].reset_index(drop=True)
    test_dfs[i] = sensor_df.iloc[n:].reset_index(drop=True)

train_dfs[1]

,temperature,label
0,47.088848,0.0
1,47.939854,0.0
2,43.477069,0.0
3,44.807002,0.0
4,49.098493,0.0
...,...,...
17275,49.537236,0.0
17276,47.045929,0.0
17277,48.019815,0.0
17278,44.587954,0.0


Convert train/test to sequences

In [ ]:
# number of past timesteps the TCN sees
seq_length = 20

# creates sequences based on data with window size == seq_length
def create_sequences(df, seq_length=20):
    '''
    Sequences are needed for giving the model information about history, since feed-forward networks do not know the order of the data.
    TCN's expect input of shape [batch_size, num_features, sequence_length]
    
    Args:
        df: time-series data
        seq_length: length of each sequence
    
    Returns:
        X (shape: [num_samples, seq_length, num_inputs]), y (shape: [num_samples, num_inputs])
    '''
    X, y = [], []
    values = df['temperature'].values
    labels = df['label'].values
    
    X = np.lib.stride_tricks.sliding_window_view(values, window_shape=seq_length)
    X = X[:-1]
    y = labels[seq_length:]
    
    X = torch.from_numpy(X).unsqueeze(-1) 
    y = torch.from_numpy(y).unsqueeze(-1)
    
    return X, y


X_train, y_train = create_sequences(train_dfs[1], seq_length)
X_test, y_test = create_sequences(test_dfs[1], seq_length)

torch.Size([17260, 1])
torch.Size([4301, 1])


Build CNN

In [ ]:
model = TCN(
    num_inputs=1,
    output_channel=1,
    output_activation=1,
    num_channels=[32, 32, 64, 64],
    kernel_size=4,
    dropout=0.1,
    use_skip_connections=True,
)

Training

Testing